In [5]:
from typing import Dict, List, Tuple, Optional
from environment import Environment, State, id_rc
from search import dijkstra
from metaheuristica import ga_main, decode
from csp import csp_select_pois_by_battery_memory

In [6]:
def reconstruct_path(prev: Dict[int, Optional[int]], start: int, goal: int, width: int) -> Optional[List[Tuple[int,int]]]:
    if prev.get(goal) is None and start != goal:
        return None
    path_ids = []
    cur = goal
    while cur is not None:
        path_ids.append(cur)
        if cur == start:
            break
        cur = prev[cur]
    path_ids.reverse()
    return [id_rc(n, width) for n in path_ids]

In [7]:
def compute_dist_matrix(env: Environment, relevant: List[int]) -> Tuple[Dict[int, Dict[int, Optional[float]]], Dict[int, Dict[int, Optional[List[Tuple[int,int]]]]]]:
    dist_matrix: Dict[int, Dict[int, Optional[float]]] = {}
    path_matrix: Dict[int, Dict[int, Optional[List[Tuple[int,int]]]]] = {}
    for u in relevant:
        dist, prev = dijkstra(env, env.base_id, u)
        print(dist)
        dist_matrix[u] = {}
        path_matrix[u] = {}
        # for v in relevant:
        #     if dist.get(v, float('inf')) == float('inf'):
        #         dist_matrix[u][v] = None
        #         path_matrix[u][v] = None
        #     else:
        #         dist_matrix[u][v] = dist[v]
        #         path_matrix[u][v] = reconstruct_path(prev, u, v, env.w)
    return dist_matrix, path_matrix

In [8]:
def get_candidate_nodes(env: 'Environment') -> List[int]:
    candidates = []
   
    for nid, node in env.nodes.items():
        if getattr(node, "terrain", "") == "plano":
            id=id_rc(nid, 5)
            candidates.append(id)
    return candidates
